# Fierce Research ATR Position Sizing For Stocks


Sources: 
- For ATR: https://www.investopedia.com/terms/a/atr.asp
- For Position sizing using 2x ATR: https://www.thebalance.com/how-average-true-range-atr-can-improve-trading-4154923

#### Import Packages

In [269]:
import numpy as np
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

#### Import Data

In [270]:
import eikon as tr
tr.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

stock = 'AEM.TO'

df1, e= tr.get_data([stock],['tr.priceclose.date','tr.priceclose','tr.open','tr.high','tr.low'],
                       {'SDate':-50,'EDate':0,'Frq':'D'})

df1['Date'] = pd.to_datetime(df1['Date'])
df1.set_index('Date', inplace=True) 
df1.index = df1.index.map(lambda t: t.strftime('%Y-%m-%d'))
df1.columns = map(str.lower, df1.columns)
df1 = df1.drop_duplicates()
df1 = df1.round(2)
df1.tail()

,instrument,price close,price open,price high,price low
2020-08-04,AEM.TO,108.62,104.88,108.75,103.44
2020-08-05,AEM.TO,110.83,109.96,111.59,109.35
2020-08-06,AEM.TO,111.90,112.00,112.37,109.65
2020-08-07,AEM.TO,110.10,109.74,112.08,109.30
2020-08-10,AEM.TO,109.38,110.78,112.10,109.07


#### True Range

The TR for a given trading period is the greatest of the following:

- Current high minus the previous close
- Current low minus the previous close
- Current high minus the current low

In [271]:
df1['price close shifted'] = df1['price close'].shift(1)

In [272]:
df1['high minus low'] = df1['price high']-df1['price low']

In [273]:
df1['low minus previous close'] = df1['price low']-df1['price close shifted']

In [274]:
df1['high minus previous close'] = df1['price high']-df1['price close shifted']

In [281]:
df1['true range'] = abs(df1[['high minus low','low minus previous close','high minus previous close']]).max(axis=1)

#### Average True Range

In [276]:
df1['atr'] = (df1['high minus low'].rolling(13).sum()+df1['high minus low'])/14
df1.tail(2).round(2)

,instrument,price close,price open,price high,price low,price close shifted,high minus low,low minus previous close,high minus previous close,true range,atr
2020-08-07,AEM.TO,110.10,109.74,112.08,109.30,111.9,2.78,-2.60,0.18,2.78,3.61
2020-08-10,AEM.TO,109.38,110.78,112.10,109.07,110.1,3.03,-1.03,2.00,3.03,3.59


In [277]:
df1['2x atr'] = df1['atr']*2

#### % Of Capital At Risk

In [278]:
df1['$ at risk'] = (.002*100000)

In [279]:
df1['recommended position size'] = df1['$ at risk']/df1['2x atr']

#### Go To Market Position Size

In [280]:
df1['recommended position size'][-1].round()

28.0

Good luck with the market! Guillaume, Fierce Research